Connected to .venv (Python 3.10.16)

In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import SAITS
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import shap
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
import optuna 
from optuna.visualization import plot_optimization_history



from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/Newdataframeforpypots.parquet")

len(df)

2025-05-13 20:19:47 [WARNING]: ‼️ PyPOTS Ecosystem configuration file does not exist.
2025-05-13 20:19:47 [INFO]: Wrote new configs to config.ini successfully.
2025-05-13 20:19:47 [INFO]: 💫 Initialized PyPOTS Ecosystem configuration file /home/ec2-user/.pypots/config.ini successfully.
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



119439

In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import SAITS
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import shap
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
import optuna 
from optuna.visualization import plot_optimization_history



from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/Newdataframeforpypots.parquet")

len(df)

#current length of the dataframe is 119439

# Check nan values in each column
for col in df.columns:
    print(f"Column {col} has {df[col].isna().sum()} NaN values")
    missing_rate=df[col].isna().sum()/len(df[col])
    print(f"Column {col} has {missing_rate} Missing_rate")


#Try with smaller dataset, size 4000
##SAMPLE the percengtage of the dataset, df.sample (averagely pick samples)
#not df.sample cuz it will randomly select 
original_size=len(df)
desired_fraction=0.10
step=int(1/desired_fraction) #step_size=10 (sample every 10th minute)

#Systematic sampling: Start at a random offset to avoid bias 
start=np.random.randint(0,step) #Random start between 0-9
df1=df.iloc[start::step].reset_index(drop=True)

print(f"Original size:{len(df)}, Sampled size: {len(df1)}")


#df1=df[:10000]


# Custom Dataset class
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Data processing code
sensor_cols = [col for col in df1.columns if col != "time"]
data = df1[sensor_cols].values

#¤get feature names for printing mae later 
feature_names=df1[sensor_cols].columns.tolist()

## Convert data to 3D arrays of shape n_samples, n_timesteps, n_features, X_ori refers to the original data without missing values 
## Reconstruct all columns simultaneously  #num_features: 119
n_features = data.shape[1]  # exclude the time column
n_steps = 1 #60 (was 60 previously) #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data.shape[0] // n_steps 



# Reshape to (n_samples // n_steps, n_steps, n_features)
#data_reshaped = data.reshape((n_samples, n_steps, n_features))
data_reshaped=data[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)
print(f"Reshaped data:{data.shape}")



##Normalization is important because of the nature of mse calculation of saits, columns with large 
#values dominate the loss, making metrics meaningless. SAITS computes MSE/MAE column-wise and averages 
#them across all columns 
#  Apply minmax scaler here 
#normalize each feature independently
scalers={}
data_normalized=data.copy()

data_normalized = np.zeros_like(data_reshaped)  # Initialize the normalized data array
scalers = {}

for i in range(data_reshaped.shape[2]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Flatten timesteps and samples for scaling
    data_normalized[:, :, i] = scaler.fit_transform(data_reshaped[:, :, i].reshape(-1, 1)).reshape(data_reshaped.shape[0], data_reshaped.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later



#Split into train, test, val

train_size = int(0.6 * len(data_normalized))
val_size = int(0.2 * len(data_normalized))
test_size = len(data_normalized) - train_size - val_size

train_data = data_normalized[:train_size]
val_data = data_normalized[train_size:train_size + val_size]
test_data= data_normalized[train_size + val_size:]

#Optional: Artificially mask. Mask 20% of the data (MIT part)
def mcar_f(X, mask_ratio=0.2):
    """Apply MCAR only to observed values."""
    observed_mask=~np.isnan(X) #find observed positions
    artificial_mask=mcar(X,mask_ratio).astype(bool) #generate MCAR mask, cast to boolean
    #combine masks 
    combined_mask=observed_mask & artificial_mask

    #Apply masking
    X_masked=X.copy()
    X_masked[combined_mask]=np.nan
    return X_masked,combined_mask


#Use mcar on validation data 
val_X_masked, val_mask =mcar_f(val_data)
val_X_ori=val_data.copy() #Ground truth

test_X_masked, test_mask =mcar_f(test_data)
test_X_ori=test_data.copy() #ground truth

#?? Problem: Can't have the best input for testing
#1.Create synthetic test_data cuz if I drop nan values for test set, there's basically nothing left
#synthetic_data=np.random.randn(n_samples,n_steps,n_features)
#test_X_masked,test_mask=mcar_f(synthetic_data)
#test_X_ori=synthetic_data.copy() #Ground truth

# 2, Ensure no NaN values in synthetic data
#test_X_masked = np.nan_to_num(test_X_masked, nan=np.nanmean(test_X_masked))
#test_X_ori = np.nan_to_num(test_X_ori, nan=np.nanmean(test_X_ori))


#3. Set aside a part of dataset for test_set
#df_test=df1[6000:8000]  #Even with 10000 rows of dataset fot testing, after dropping nan value in rows, there is none left 
#2. Drop nan rows 
#f_test=df_test.dropna(axis=0,how='all')
#print("Size of the test set:", len(df_test)) 
#data_test=df_test[sensor_cols].values



#n_features = data_test.shape[1]  # exclude the time column
#n_steps = 60 #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
#n_samples = data_test.shape[0] // n_steps 
#Reshape and apply mcar to the test set 
# #data_test=data_test[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)

#data_test_normalized=np.zeros_like(data_test)  # Initialize the normalized data array
#for i in range(data_test.shape[2]):
  # scaler = MinMaxScaler(feature_range=(0, 1))
    # Flatten timesteps and samples for scaling
  # data_test_normalized[:, :, i] = scaler.fit_transform(data_test[:, :, i].reshape(-1, 1)).reshape(data_test.shape[0], data_test.shape[1])
  # scalers[i] = scaler  # Save scalers to inverse-transform later




class Config:
    no_cuda = False
    no_mps = False
    seed = 1

args=Config()

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)


args.cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

if args.cuda:
    device = torch.device("cuda")
    print("Using CUDA")
elif use_mps:
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")


#MLflow set up
mlflow.set_tracking_uri("http://localhost:5000")
client = mlflow.tracking.MlflowClient()
mlflow.set_experiment("SAITS")
SAITS_run_name = "SAITS"

with mlflow.start_run(run_name=SAITS_run_name) as run:
# initialize the model (from example)
    saits = SAITS(
        n_steps=data_normalized.shape[1],
        n_features=data_normalized.shape[2],
        n_layers=2, #deep network(4) for long sequences, here is revised for 2 since it always shows cuda error 
        d_model=512,  #d_modle must equal n_heads * d_k
        d_ffn=512,
        n_heads=8,
        d_k=64,
        d_v=64,
        dropout=0.1,
        attn_dropout=0.1,
        diagonal_attention_mask=True,  # otherwise the original self-attention mechanism will be applied
        ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
       # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
        MIT_weight=1,
        batch_size=5, #try with 5 to see if it works, was 4. 
        # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
        epochs=10, #try with 10 to see if it runs
        # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
        # You can leave it to defualt as None to disable early stopping.
        patience=4, #initially was 3, tried to increase to see more possibilities 
        # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
        # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
        optimizer=Adam(lr=1e-3),
        # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
        # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
        # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
        num_workers=0,
        # just leave it to default as None, PyPOTS will automatically assign the best device for you.
        # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
        device=device, 
        # set the path for saving tensorboard and trained model files 
        saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model",
        # only save the best model after training finished.
        # You can also set it as "better" to save models performing better ever during training.
        model_saving_strategy="best",
)


# Log model parameters
mlflow.log_param("n_steps", data_normalized.shape[1])
mlflow.log_param("n_features", data_normalized.shape[2])
mlflow.log_param("batch_size", 5)
mlflow.log_param("epochs", 10)
mlflow.log_param("patience", 4)


# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
#use original missigness for trainig, use the mcar masked points as ground truth
saits.fit(train_set={"X": train_data}, val_set={"X": val_X_masked , "X_ori":val_X_ori })


##drop null values in test set 

# Check for NaN values across the entire array
#nan_mask = np.isnan(test_X).any(axis=(1, 2))

# Filter out samples (rows) that contain NaN values
#test_X_clean = test_X[~nan_mask]
#test_X_ori = test_X_clean
#test_X_masked, test_mask = mcar_f(test_X_clean)

# the testing stage, impute the originally-missing values and artificially-missing values in the test set
# Convert the numpy array to a dictionary
test_set_dict = {"X": test_X_masked}
test_imputation = saits.predict(test_set_dict)

#change back to array for applying inverse scale later
test_imputation_array = test_imputation["imputation"]


#clear cuda after prediction if on cuda

# Ensure saits_imputation and test_X are numpy arrays
#if not isinstance(saits_imputation, np.ndarray):
    #saits_imputation = np.array(saits_imputation)
#if not isinstance(test_X, np.ndarray):
     #test_X = np.array(test_X)

#Inverse Scale
def inverse_scale(imputation, scalers):
    n_features = imputation.shape[2]
    imputation_denorm = np.empty_like(imputation)
    
    for i in range(n_features):
        imputation_denorm[:, :, i] = scalers[i].inverse_transform(imputation[:, :, i].reshape(-1, 1)).reshape(imputation.shape[0], imputation.shape[1])
    
    return imputation_denorm  # Move the return statement outside the loop

    
#Apply function to the dataset 
test_imputation_denorm=inverse_scale(test_imputation_array,scalers)
test_X_ori_denorm=inverse_scale(test_X_ori,scalers)

if args.cuda:
    torch.cuda.empty_cache() 


#Calculate metrics per-feature: mean absolute error on the ground truth (artificially-missing values)
mae_per_feature=[]
for i in range(n_features):
    #Extract imputation and ground truth for feature i
    imputation_i=test_imputation_denorm[:,:,i]
    ground_truth_i=test_X_ori_denorm[:,:,i]
    mask_i=test_mask[:,:,i]
    # Check for NaN values
    if np.isnan(imputation_i).any() or np.isnan(ground_truth_i).any():
        print(f"NaN values detected in feature {i}")
        continue  # Skip this feature if NaN values are found
    #Filter only artificially masked positions
    mae_i=calc_mae(imputation_i,ground_truth_i,mask_i)
    mae_per_feature.append(mae_i)
    print(f"MAE for {feature_names[i]}: {mae_i:.4f}")
    mlflow.log_metric(f"MAE_{feature_names[i]}", mae_i)


# calculate average MAE 
avg_mae=np.mean(mae_per_feature)
print(f"Testing mean absolute error: {avg_mae:.4f}")
mlflow.log_metric("avg_mae", avg_mae)


##Current error is on the cuda 
#test memory
#def test_memory(in_size=100, out_size=10,hidden_size=100,optimizer_type=torch.optim.Adam, batch_size=1, device=0):
   # sample_input=torch.randn(batch_size,in_size)
  #  model=saits,
   # optimizer=optimizer_type(model.parameters(),lr=1e-3)
   # print("Beginning mem:",torch.cuda.memory_allocated(device))
   # model.to(device)
   # print()
   # for i in range(3):
   #     print("Iteration",i)
   #     a=torch.cuda.memory_allocated(device)
   #     out=model(sample_input.to(device)).sum()
   #     b=torch.cuda.memory_allocated(device)
   #     print("2-After forward pass", torch.cuda.memory_allocated(device))
   #     print("2-Memory consumed by forward pass", b-a)
   #     out.backward()
   #     print("3-After backward pass", torch.cuda.memory_allocated(device))
   #     optimizer.step()
   #     print("4-After optimizer step", torch.cuda.memory_allocated(device))

##To do: log mlflow, 

Column time has 0 NaN values
Column time has 0.0 Missing_rate
Column fr_eng has 0 NaN values
Column fr_eng has 0.0 Missing_rate
Column te_exh_cyl_out__0 has 73 NaN values
Column te_exh_cyl_out__0 has 0.0006111906496203082 Missing_rate
Column pd_air_ic__0 has 73 NaN values
Column pd_air_ic__0 has 0.0006111906496203082 Missing_rate
Column pr_exh_turb_out__0 has 119439 NaN values
Column pr_exh_turb_out__0 has 1.0 Missing_rate
Column te_air_ic_out__0 has 73 NaN values
Column te_air_ic_out__0 has 0.0006111906496203082 Missing_rate
Column te_seawater has 73 NaN values
Column te_seawater has 0.0006111906496203082 Missing_rate
Column te_air_comp_in_a__0 has 119439 NaN values
Column te_air_comp_in_a__0 has 1.0 Missing_rate
Column te_air_comp_in_b__0 has 119439 NaN values
Column te_air_comp_in_b__0 has 1.0 Missing_rate
Column fr_tc__0 has 119439 NaN values
Column fr_tc__0 has 1.0 Missing_rate
Column pr_baro has 73 NaN values
Column pr_baro has 0.0006111906496203082 Missing_rate
Column pd_air_ic_

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

Using CPU


MlflowException: API request to http://localhost:5000/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=SAITS (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f1b5015c760>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import SAITS
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import shap
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
import optuna 
from optuna.visualization import plot_optimization_history



from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/Newdataframeforpypots.parquet")

len(df)

#current length of the dataframe is 119439

# Check nan values in each column
for col in df.columns:
    print(f"Column {col} has {df[col].isna().sum()} NaN values")
    missing_rate=df[col].isna().sum()/len(df[col])
    print(f"Column {col} has {missing_rate} Missing_rate")


#Try with smaller dataset, size 4000
##SAMPLE the percengtage of the dataset, df.sample (averagely pick samples)
#not df.sample cuz it will randomly select 
original_size=len(df)
desired_fraction=0.10
step=int(1/desired_fraction) #step_size=10 (sample every 10th minute)

#Systematic sampling: Start at a random offset to avoid bias 
start=np.random.randint(0,step) #Random start between 0-9
df1=df.iloc[start::step].reset_index(drop=True)

print(f"Original size:{len(df)}, Sampled size: {len(df1)}")


#df1=df[:10000]


# Custom Dataset class
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Data processing code
sensor_cols = [col for col in df1.columns if col != "time"]
data = df1[sensor_cols].values

#¤get feature names for printing mae later 
feature_names=df1[sensor_cols].columns.tolist()

## Convert data to 3D arrays of shape n_samples, n_timesteps, n_features, X_ori refers to the original data without missing values 
## Reconstruct all columns simultaneously  #num_features: 119
n_features = data.shape[1]  # exclude the time column
n_steps = 1 #60 (was 60 previously) #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data.shape[0] // n_steps 



# Reshape to (n_samples // n_steps, n_steps, n_features)
#data_reshaped = data.reshape((n_samples, n_steps, n_features))
data_reshaped=data[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)
print(f"Reshaped data:{data.shape}")



##Normalization is important because of the nature of mse calculation of saits, columns with large 
#values dominate the loss, making metrics meaningless. SAITS computes MSE/MAE column-wise and averages 
#them across all columns 
#  Apply minmax scaler here 
#normalize each feature independently
scalers={}
data_normalized=data.copy()

data_normalized = np.zeros_like(data_reshaped)  # Initialize the normalized data array
scalers = {}

for i in range(data_reshaped.shape[2]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Flatten timesteps and samples for scaling
    data_normalized[:, :, i] = scaler.fit_transform(data_reshaped[:, :, i].reshape(-1, 1)).reshape(data_reshaped.shape[0], data_reshaped.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later



#Split into train, test, val

train_size = int(0.6 * len(data_normalized))
val_size = int(0.2 * len(data_normalized))
test_size = len(data_normalized) - train_size - val_size

train_data = data_normalized[:train_size]
val_data = data_normalized[train_size:train_size + val_size]
test_data= data_normalized[train_size + val_size:]

#Optional: Artificially mask. Mask 20% of the data (MIT part)
def mcar_f(X, mask_ratio=0.2):
    """Apply MCAR only to observed values."""
    observed_mask=~np.isnan(X) #find observed positions
    artificial_mask=mcar(X,mask_ratio).astype(bool) #generate MCAR mask, cast to boolean
    #combine masks 
    combined_mask=observed_mask & artificial_mask

    #Apply masking
    X_masked=X.copy()
    X_masked[combined_mask]=np.nan
    return X_masked,combined_mask


#Use mcar on validation data 
val_X_masked, val_mask =mcar_f(val_data)
val_X_ori=val_data.copy() #Ground truth

test_X_masked, test_mask =mcar_f(test_data)
test_X_ori=test_data.copy() #ground truth

#?? Problem: Can't have the best input for testing
#1.Create synthetic test_data cuz if I drop nan values for test set, there's basically nothing left
#synthetic_data=np.random.randn(n_samples,n_steps,n_features)
#test_X_masked,test_mask=mcar_f(synthetic_data)
#test_X_ori=synthetic_data.copy() #Ground truth

# 2, Ensure no NaN values in synthetic data
#test_X_masked = np.nan_to_num(test_X_masked, nan=np.nanmean(test_X_masked))
#test_X_ori = np.nan_to_num(test_X_ori, nan=np.nanmean(test_X_ori))


#3. Set aside a part of dataset for test_set
#df_test=df1[6000:8000]  #Even with 10000 rows of dataset fot testing, after dropping nan value in rows, there is none left 
#2. Drop nan rows 
#f_test=df_test.dropna(axis=0,how='all')
#print("Size of the test set:", len(df_test)) 
#data_test=df_test[sensor_cols].values



#n_features = data_test.shape[1]  # exclude the time column
#n_steps = 60 #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
#n_samples = data_test.shape[0] // n_steps 
#Reshape and apply mcar to the test set 
# #data_test=data_test[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)

#data_test_normalized=np.zeros_like(data_test)  # Initialize the normalized data array
#for i in range(data_test.shape[2]):
  # scaler = MinMaxScaler(feature_range=(0, 1))
    # Flatten timesteps and samples for scaling
  # data_test_normalized[:, :, i] = scaler.fit_transform(data_test[:, :, i].reshape(-1, 1)).reshape(data_test.shape[0], data_test.shape[1])
  # scalers[i] = scaler  # Save scalers to inverse-transform later




class Config:
    no_cuda = False
    no_mps = False
    seed = 1

args=Config()

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)


args.cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

if args.cuda:
    device = torch.device("cuda")
    print("Using CUDA")
elif use_mps:
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")


#MLflow set up
mlflow.set_tracking_uri("http://localhost:5000")
client = mlflow.tracking.MlflowClient()
mlflow.set_experiment("SAITS")
SAITS_run_name = "SAITS"

with mlflow.start_run(run_name=SAITS_run_name) as run:
# initialize the model (from example)
    saits = SAITS(
        n_steps=data_normalized.shape[1],
        n_features=data_normalized.shape[2],
        n_layers=2, #deep network(4) for long sequences, here is revised for 2 since it always shows cuda error 
        d_model=512,  #d_modle must equal n_heads * d_k
        d_ffn=512,
        n_heads=8,
        d_k=64,
        d_v=64,
        dropout=0.1,
        attn_dropout=0.1,
        diagonal_attention_mask=True,  # otherwise the original self-attention mechanism will be applied
        ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
       # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
        MIT_weight=1,
        batch_size=5, #try with 5 to see if it works, was 4. 
        # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
        epochs=10, #try with 10 to see if it runs
        # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
        # You can leave it to defualt as None to disable early stopping.
        patience=4, #initially was 3, tried to increase to see more possibilities 
        # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
        # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
        optimizer=Adam(lr=1e-3),
        # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
        # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
        # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
        num_workers=0,
        # just leave it to default as None, PyPOTS will automatically assign the best device for you.
        # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
        device=device, 
        # set the path for saving tensorboard and trained model files 
        saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model",
        # only save the best model after training finished.
        # You can also set it as "better" to save models performing better ever during training.
        model_saving_strategy="best",
)


# Log model parameters
mlflow.log_param("n_steps", data_normalized.shape[1])
mlflow.log_param("n_features", data_normalized.shape[2])
mlflow.log_param("batch_size", 5)
mlflow.log_param("epochs", 10)
mlflow.log_param("patience", 4)


# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
#use original missigness for trainig, use the mcar masked points as ground truth
saits.fit(train_set={"X": train_data}, val_set={"X": val_X_masked , "X_ori":val_X_ori })


##drop null values in test set 

# Check for NaN values across the entire array
#nan_mask = np.isnan(test_X).any(axis=(1, 2))

# Filter out samples (rows) that contain NaN values
#test_X_clean = test_X[~nan_mask]
#test_X_ori = test_X_clean
#test_X_masked, test_mask = mcar_f(test_X_clean)

# the testing stage, impute the originally-missing values and artificially-missing values in the test set
# Convert the numpy array to a dictionary
test_set_dict = {"X": test_X_masked}
test_imputation = saits.predict(test_set_dict)

#change back to array for applying inverse scale later
test_imputation_array = test_imputation["imputation"]


#clear cuda after prediction if on cuda

# Ensure saits_imputation and test_X are numpy arrays
#if not isinstance(saits_imputation, np.ndarray):
    #saits_imputation = np.array(saits_imputation)
#if not isinstance(test_X, np.ndarray):
     #test_X = np.array(test_X)

#Inverse Scale
def inverse_scale(imputation, scalers):
    n_features = imputation.shape[2]
    imputation_denorm = np.empty_like(imputation)
    
    for i in range(n_features):
        imputation_denorm[:, :, i] = scalers[i].inverse_transform(imputation[:, :, i].reshape(-1, 1)).reshape(imputation.shape[0], imputation.shape[1])
    
    return imputation_denorm  # Move the return statement outside the loop

    
#Apply function to the dataset 
test_imputation_denorm=inverse_scale(test_imputation_array,scalers)
test_X_ori_denorm=inverse_scale(test_X_ori,scalers)

if args.cuda:
    torch.cuda.empty_cache() 


#Calculate metrics per-feature: mean absolute error on the ground truth (artificially-missing values)
mae_per_feature=[]
for i in range(n_features):
    #Extract imputation and ground truth for feature i
    imputation_i=test_imputation_denorm[:,:,i]
    ground_truth_i=test_X_ori_denorm[:,:,i]
    mask_i=test_mask[:,:,i]
    # Check for NaN values
    if np.isnan(imputation_i).any() or np.isnan(ground_truth_i).any():
        print(f"NaN values detected in feature {i}")
        continue  # Skip this feature if NaN values are found
    #Filter only artificially masked positions
    mae_i=calc_mae(imputation_i,ground_truth_i,mask_i)
    mae_per_feature.append(mae_i)
    print(f"MAE for {feature_names[i]}: {mae_i:.4f}")
    mlflow.log_metric(f"MAE_{feature_names[i]}", mae_i)


# calculate average MAE 
avg_mae=np.mean(mae_per_feature)
print(f"Testing mean absolute error: {avg_mae:.4f}")
mlflow.log_metric("avg_mae", avg_mae)


##Current error is on the cuda 
#test memory
#def test_memory(in_size=100, out_size=10,hidden_size=100,optimizer_type=torch.optim.Adam, batch_size=1, device=0):
   # sample_input=torch.randn(batch_size,in_size)
  #  model=saits,
   # optimizer=optimizer_type(model.parameters(),lr=1e-3)
   # print("Beginning mem:",torch.cuda.memory_allocated(device))
   # model.to(device)
   # print()
   # for i in range(3):
   #     print("Iteration",i)
   #     a=torch.cuda.memory_allocated(device)
   #     out=model(sample_input.to(device)).sum()
   #     b=torch.cuda.memory_allocated(device)
   #     print("2-After forward pass", torch.cuda.memory_allocated(device))
   #     print("2-Memory consumed by forward pass", b-a)
   #     out.backward()
   #     print("3-After backward pass", torch.cuda.memory_allocated(device))
   #     optimizer.step()
   #     print("4-After optimizer step", torch.cuda.memory_allocated(device))

##To do: log mlflow, 

Column time has 0 NaN values
Column time has 0.0 Missing_rate
Column fr_eng has 0 NaN values
Column fr_eng has 0.0 Missing_rate
Column te_exh_cyl_out__0 has 73 NaN values
Column te_exh_cyl_out__0 has 0.0006111906496203082 Missing_rate
Column pd_air_ic__0 has 73 NaN values
Column pd_air_ic__0 has 0.0006111906496203082 Missing_rate
Column pr_exh_turb_out__0 has 119439 NaN values
Column pr_exh_turb_out__0 has 1.0 Missing_rate
Column te_air_ic_out__0 has 73 NaN values
Column te_air_ic_out__0 has 0.0006111906496203082 Missing_rate
Column te_seawater has 73 NaN values
Column te_seawater has 0.0006111906496203082 Missing_rate
Column te_air_comp_in_a__0 has 119439 NaN values
Column te_air_comp_in_a__0 has 1.0 Missing_rate
Column te_air_comp_in_b__0 has 119439 NaN values
Column te_air_comp_in_b__0 has 1.0 Missing_rate
Column fr_tc__0 has 119439 NaN values
Column fr_tc__0 has 1.0 Missing_rate
Column pr_baro has 73 NaN values
Column pr_baro has 0.0006111906496203082 Missing_rate
Column pd_air_ic_

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

🏃 View run SAITS at: http://localhost:5000/#/experiments/1/runs/dd2bf2404745444c8c0e8b3b9e4c73f7
🧪 View experiment at: http://localhost:5000/#/experiments/1


2025-05-13 21:29:29 [ERROR]: ❌ Exception: `predictions` mustn't contain NaN values, but detected NaN in it


RuntimeError: Training got interrupted. Model was not trained. Please investigate the error printed above.

In [ ]:

if np.isnan(train_data).any():
    raise ValueError("NaN values detected in training data")

# Check for NaN values in validation data
if np.isnan(val_X_masked).any() or np.isnan(val_X_ori).any():
    raise ValueError("NaN values detected in validation data")

# Check for NaN values in test data
if np.isnan(test_X_masked).any() or np.isnan(test_X_ori).any():
    raise ValueError("NaN values detected in test data")


SyntaxError: invalid non-printable character U+00A0 (<ipython-input-4-491befc0bc6f>, line 2)

In [ ]:
if np.isnan(train_data).any():
    raise ValueError("NaN values detected in training data")

# Check for NaN values in validation data
if np.isnan(val_X_masked).any() or np.isnan(val_X_ori).any():
    raise ValueError("NaN values detected in validation data")

# Check for NaN values in test data
if np.isnan(test_X_masked).any() or np.isnan(test_X_ori).any():
    raise ValueError("NaN values detected in test data")


SyntaxError: invalid non-printable character U+00A0 (<ipython-input-5-975a705d4f20>, line 2)

In [ ]:

if np.isnan(train_data).any():
    raise ValueError("NaN values detected in training data")

# Check for NaN values in validation data
if np.isnan(val_X_masked).any() or np.isnan(val_X_ori).any():
    raise ValueError("NaN values detected in validation data")

# Check for NaN values in test data
if np.isnan(test_X_masked).any() or np.isnan(test_X_ori).any():
    raise ValueError("NaN values detected in test data")


SyntaxError: invalid non-printable character U+00A0 (<ipython-input-6-491befc0bc6f>, line 2)

In [ ]:
np.isnan(train_data).any()

True

In [ ]:
print(df1)

                     time    fr_eng  te_exh_cyl_out__0  pd_air_ic__0  \
0     2023-10-01 00:05:00  0.734716             510.15         450.0   
1     2023-10-01 00:15:00  0.733886             510.15         430.0   
2     2023-10-01 00:25:00  0.733876             511.15         460.0   
3     2023-10-01 00:35:00  0.732363             511.15         430.0   
4     2023-10-01 00:45:00  0.734891             511.15         480.0   
...                   ...       ...                ...           ...   
11939 2024-01-31 05:46:00  0.536374             412.15         450.0   
11940 2024-01-31 05:56:00  0.529955             455.15         460.0   
11941 2024-01-31 06:06:00  0.348162             417.15         300.0   
11942 2024-01-31 06:16:00  0.350018             406.15         280.0   
11943 2024-01-31 06:26:00  0.349860             400.15         270.0   

       pr_exh_turb_out__0  te_air_ic_out__0  te_seawater  te_air_comp_in_a__0  \
0                     NaN            308.65       273.

In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import SAITS
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import shap
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
import optuna 
from optuna.visualization import plot_optimization_history



from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/Newdataframeforpypots.parquet")

len(df)

#current length of the dataframe is 119439

# Check nan values in each column
for col in df.columns:
    print(f"Column {col} has {df[col].isna().sum()} NaN values")
    missing_rate=df[col].isna().sum()/len(df[col])
    print(f"Column {col} has {missing_rate} Missing_rate")


#Try with smaller dataset, size 4000
##SAMPLE the percengtage of the dataset, df.sample (averagely pick samples)
#not df.sample cuz it will randomly select 
original_size=len(df)
desired_fraction=0.10
step=int(1/desired_fraction) #step_size=10 (sample every 10th minute)

#Systematic sampling: Start at a random offset to avoid bias 
start=np.random.randint(0,step) #Random start between 0-9
df1=df.iloc[start::step].reset_index(drop=True)

print(f"Original size:{len(df)}, Sampled size: {len(df1)}")


#df1=df[:10000]


# Custom Dataset class
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Data processing code
sensor_cols = [col for col in df1.columns if col != "time"]
data = df1[sensor_cols].values

#¤get feature names for printing mae later 
feature_names=df1[sensor_cols].columns.tolist()

## Convert data to 3D arrays of shape n_samples, n_timesteps, n_features, X_ori refers to the original data without missing values 
## Reconstruct all columns simultaneously  #num_features: 119
n_features = data.shape[1]  # exclude the time column
n_steps = 30 #60 (was 60 previously) #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data.shape[0] // n_steps 



# Reshape to (n_samples // n_steps, n_steps, n_features)
#data_reshaped = data.reshape((n_samples, n_steps, n_features))
data_reshaped=data[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)
print(f"Reshaped data:{data.shape}")



##Normalization is important because of the nature of mse calculation of saits, columns with large 
#values dominate the loss, making metrics meaningless. SAITS computes MSE/MAE column-wise and averages 
#them across all columns 
#  Apply minmax scaler here 
#normalize each feature independently
scalers={}
data_normalized=data.copy()

data_normalized = np.zeros_like(data_reshaped)  # Initialize the normalized data array
scalers = {}

for i in range(data_reshaped.shape[2]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Flatten timesteps and samples for scaling
    data_normalized[:, :, i] = scaler.fit_transform(data_reshaped[:, :, i].reshape(-1, 1)).reshape(data_reshaped.shape[0], data_reshaped.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later



#Split into train, test, val

train_size = int(0.6 * len(data_normalized))
val_size = int(0.2 * len(data_normalized))
test_size = len(data_normalized) - train_size - val_size

train_data = data_normalized[:train_size]
val_data = data_normalized[train_size:train_size + val_size]
test_data= data_normalized[train_size + val_size:]

#Optional: Artificially mask. Mask 20% of the data (MIT part)
def mcar_f(X, mask_ratio=0.2):
    """Apply MCAR only to observed values."""
    observed_mask=~np.isnan(X) #find observed positions
    artificial_mask=mcar(X,mask_ratio).astype(bool) #generate MCAR mask, cast to boolean
    #combine masks 
    combined_mask=observed_mask & artificial_mask

    #Apply masking
    X_masked=X.copy()
    X_masked[combined_mask]=np.nan
    return X_masked,combined_mask


#Use mcar on validation data 
val_X_masked, val_mask =mcar_f(val_data)
val_X_ori=val_data.copy() #Ground truth

test_X_masked, test_mask =mcar_f(test_data)
test_X_ori=test_data.copy() #ground truth

#?? Problem: Can't have the best input for testing
#1.Create synthetic test_data cuz if I drop nan values for test set, there's basically nothing left
#synthetic_data=np.random.randn(n_samples,n_steps,n_features)
#test_X_masked,test_mask=mcar_f(synthetic_data)
#test_X_ori=synthetic_data.copy() #Ground truth

# 2, Ensure no NaN values in synthetic data
#test_X_masked = np.nan_to_num(test_X_masked, nan=np.nanmean(test_X_masked))
#test_X_ori = np.nan_to_num(test_X_ori, nan=np.nanmean(test_X_ori))


#3. Set aside a part of dataset for test_set
#df_test=df1[6000:8000]  #Even with 10000 rows of dataset fot testing, after dropping nan value in rows, there is none left 
#2. Drop nan rows 
#f_test=df_test.dropna(axis=0,how='all')
#print("Size of the test set:", len(df_test)) 
#data_test=df_test[sensor_cols].values



#n_features = data_test.shape[1]  # exclude the time column
#n_steps = 60 #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
#n_samples = data_test.shape[0] // n_steps 
#Reshape and apply mcar to the test set 
# #data_test=data_test[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)

#data_test_normalized=np.zeros_like(data_test)  # Initialize the normalized data array
#for i in range(data_test.shape[2]):
  # scaler = MinMaxScaler(feature_range=(0, 1))
    # Flatten timesteps and samples for scaling
  # data_test_normalized[:, :, i] = scaler.fit_transform(data_test[:, :, i].reshape(-1, 1)).reshape(data_test.shape[0], data_test.shape[1])
  # scalers[i] = scaler  # Save scalers to inverse-transform later




class Config:
    no_cuda = False
    no_mps = False
    seed = 1

args=Config()

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)


args.cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

if args.cuda:
    device = torch.device("cuda")
    print("Using CUDA")
elif use_mps:
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")


#MLflow set up
mlflow.set_tracking_uri("http://localhost:5000")
client = mlflow.tracking.MlflowClient()
mlflow.set_experiment("SAITS")
SAITS_run_name = "SAITS"

with mlflow.start_run(run_name=SAITS_run_name) as run:
# initialize the model (from example)
    saits = SAITS(
        n_steps=data_normalized.shape[1],
        n_features=data_normalized.shape[2],
        n_layers=2, #deep network(4) for long sequences, here is revised for 2 since it always shows cuda error 
        d_model=512,  #d_modle must equal n_heads * d_k
        d_ffn=512,
        n_heads=8,
        d_k=64,
        d_v=64,
        dropout=0.1,
        attn_dropout=0.1,
        diagonal_attention_mask=True,  # otherwise the original self-attention mechanism will be applied
        ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
       # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
        MIT_weight=1,
        batch_size=5, #try with 5 to see if it works, was 4. 
        # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
        epochs=10, #try with 10 to see if it runs
        # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
        # You can leave it to defualt as None to disable early stopping.
        patience=4, #initially was 3, tried to increase to see more possibilities 
        # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
        # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
        optimizer=Adam(lr=1e-3),
        # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
        # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
        # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
        num_workers=0,
        # just leave it to default as None, PyPOTS will automatically assign the best device for you.
        # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
        device=device, 
        # set the path for saving tensorboard and trained model files 
        saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model",
        # only save the best model after training finished.
        # You can also set it as "better" to save models performing better ever during training.
        model_saving_strategy="best",
)


# Log model parameters
mlflow.log_param("n_steps", data_normalized.shape[1])
mlflow.log_param("n_features", data_normalized.shape[2])
mlflow.log_param("batch_size", 5)
mlflow.log_param("epochs", 10)
mlflow.log_param("patience", 4)


# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
#use original missigness for trainig, use the mcar masked points as ground truth
saits.fit(train_set={"X": train_data}, val_set={"X": val_X_masked , "X_ori":val_X_ori })


##drop null values in test set 

# Check for NaN values across the entire array
#nan_mask = np.isnan(test_X).any(axis=(1, 2))

# Filter out samples (rows) that contain NaN values
#test_X_clean = test_X[~nan_mask]
#test_X_ori = test_X_clean
#test_X_masked, test_mask = mcar_f(test_X_clean)

# the testing stage, impute the originally-missing values and artificially-missing values in the test set
# Convert the numpy array to a dictionary
test_set_dict = {"X": test_X_masked}
test_imputation = saits.predict(test_set_dict)

#change back to array for applying inverse scale later
test_imputation_array = test_imputation["imputation"]


#clear cuda after prediction if on cuda

# Ensure saits_imputation and test_X are numpy arrays
#if not isinstance(saits_imputation, np.ndarray):
    #saits_imputation = np.array(saits_imputation)
#if not isinstance(test_X, np.ndarray):
     #test_X = np.array(test_X)

#Inverse Scale
def inverse_scale(imputation, scalers):
    n_features = imputation.shape[2]
    imputation_denorm = np.empty_like(imputation)
    
    for i in range(n_features):
        imputation_denorm[:, :, i] = scalers[i].inverse_transform(imputation[:, :, i].reshape(-1, 1)).reshape(imputation.shape[0], imputation.shape[1])
    
    return imputation_denorm  # Move the return statement outside the loop

    
#Apply function to the dataset 
test_imputation_denorm=inverse_scale(test_imputation_array,scalers)
test_X_ori_denorm=inverse_scale(test_X_ori,scalers)

if args.cuda:
    torch.cuda.empty_cache() 


#Calculate metrics per-feature: mean absolute error on the ground truth (artificially-missing values)
mae_per_feature=[]
for i in range(n_features):
    #Extract imputation and ground truth for feature i
    imputation_i=test_imputation_denorm[:,:,i]
    ground_truth_i=test_X_ori_denorm[:,:,i]
    mask_i=test_mask[:,:,i]
    # Check for NaN values
    if np.isnan(imputation_i).any() or np.isnan(ground_truth_i).any():
        print(f"NaN values detected in feature {i}")
        continue  # Skip this feature if NaN values are found
    #Filter only artificially masked positions
    mae_i=calc_mae(imputation_i,ground_truth_i,mask_i)
    mae_per_feature.append(mae_i)
    print(f"MAE for {feature_names[i]}: {mae_i:.4f}")
    mlflow.log_metric(f"MAE_{feature_names[i]}", mae_i)


# calculate average MAE 
avg_mae=np.mean(mae_per_feature)
print(f"Testing mean absolute error: {avg_mae:.4f}")
mlflow.log_metric("avg_mae", avg_mae)


##Current error is on the cuda 
#test memory
#def test_memory(in_size=100, out_size=10,hidden_size=100,optimizer_type=torch.optim.Adam, batch_size=1, device=0):
   # sample_input=torch.randn(batch_size,in_size)
  #  model=saits,
   # optimizer=optimizer_type(model.parameters(),lr=1e-3)
   # print("Beginning mem:",torch.cuda.memory_allocated(device))
   # model.to(device)
   # print()
   # for i in range(3):
   #     print("Iteration",i)
   #     a=torch.cuda.memory_allocated(device)
   #     out=model(sample_input.to(device)).sum()
   #     b=torch.cuda.memory_allocated(device)
   #     print("2-After forward pass", torch.cuda.memory_allocated(device))
   #     print("2-Memory consumed by forward pass", b-a)
   #     out.backward()
   #     print("3-After backward pass", torch.cuda.memory_allocated(device))
   #     optimizer.step()
   #     print("4-After optimizer step", torch.cuda.memory_allocated(device))

##To do: log mlflow, 

Column time has 0 NaN values
Column time has 0.0 Missing_rate
Column fr_eng has 0 NaN values
Column fr_eng has 0.0 Missing_rate
Column te_exh_cyl_out__0 has 73 NaN values
Column te_exh_cyl_out__0 has 0.0006111906496203082 Missing_rate
Column pd_air_ic__0 has 73 NaN values
Column pd_air_ic__0 has 0.0006111906496203082 Missing_rate
Column pr_exh_turb_out__0 has 119439 NaN values
Column pr_exh_turb_out__0 has 1.0 Missing_rate
Column te_air_ic_out__0 has 73 NaN values
Column te_air_ic_out__0 has 0.0006111906496203082 Missing_rate
Column te_seawater has 73 NaN values
Column te_seawater has 0.0006111906496203082 Missing_rate
Column te_air_comp_in_a__0 has 119439 NaN values
Column te_air_comp_in_a__0 has 1.0 Missing_rate
Column te_air_comp_in_b__0 has 119439 NaN values
Column te_air_comp_in_b__0 has 1.0 Missing_rate
Column fr_tc__0 has 119439 NaN values
Column fr_tc__0 has 1.0 Missing_rate
Column pr_baro has 73 NaN values
Column pr_baro has 0.0006111906496203082 Missing_rate
Column pd_air_ic_

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

Exception: Run with UUID a5285036106e4611b162d9ad8f532b5a is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True

In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import SAITS
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import shap
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
import optuna 
from optuna.visualization import plot_optimization_history



from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/Newdataframeforpypots.parquet")

len(df)

#current length of the dataframe is 119439

# Check nan values in each column
for col in df.columns:
    print(f"Column {col} has {df[col].isna().sum()} NaN values")
    missing_rate=df[col].isna().sum()/len(df[col])
    print(f"Column {col} has {missing_rate} Missing_rate")


#Try with smaller dataset, size 4000
##SAMPLE the percengtage of the dataset, df.sample (averagely pick samples)
#not df.sample cuz it will randomly select 
original_size=len(df)
desired_fraction=0.10
step=int(1/desired_fraction) #step_size=10 (sample every 10th minute)

#Systematic sampling: Start at a random offset to avoid bias 
start=np.random.randint(0,step) #Random start between 0-9
df1=df.iloc[start::step].reset_index(drop=True)

print(f"Original size:{len(df)}, Sampled size: {len(df1)}")


#df1=df[:10000]


# Custom Dataset class
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Data processing code
sensor_cols = [col for col in df1.columns if col != "time"]
data = df1[sensor_cols].values

#¤get feature names for printing mae later 
feature_names=df1[sensor_cols].columns.tolist()

## Convert data to 3D arrays of shape n_samples, n_timesteps, n_features, X_ori refers to the original data without missing values 
## Reconstruct all columns simultaneously  #num_features: 119
n_features = data.shape[1]  # exclude the time column
n_steps = 30 #60 (was 60 previously) #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data.shape[0] // n_steps 



# Reshape to (n_samples // n_steps, n_steps, n_features)
#data_reshaped = data.reshape((n_samples, n_steps, n_features))
data_reshaped=data[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)
print(f"Reshaped data:{data.shape}")



##Normalization is important because of the nature of mse calculation of saits, columns with large 
#values dominate the loss, making metrics meaningless. SAITS computes MSE/MAE column-wise and averages 
#them across all columns 
#  Apply minmax scaler here 
#normalize each feature independently
scalers={}
data_normalized=data.copy()

data_normalized = np.zeros_like(data_reshaped)  # Initialize the normalized data array
scalers = {}

for i in range(data_reshaped.shape[2]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Flatten timesteps and samples for scaling
    data_normalized[:, :, i] = scaler.fit_transform(data_reshaped[:, :, i].reshape(-1, 1)).reshape(data_reshaped.shape[0], data_reshaped.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later



#Split into train, test, val

train_size = int(0.6 * len(data_normalized))
val_size = int(0.2 * len(data_normalized))
test_size = len(data_normalized) - train_size - val_size

train_data = data_normalized[:train_size]
val_data = data_normalized[train_size:train_size + val_size]
test_data= data_normalized[train_size + val_size:]

#Optional: Artificially mask. Mask 20% of the data (MIT part)
def mcar_f(X, mask_ratio=0.2):
    """Apply MCAR only to observed values."""
    observed_mask=~np.isnan(X) #find observed positions
    artificial_mask=mcar(X,mask_ratio).astype(bool) #generate MCAR mask, cast to boolean
    #combine masks 
    combined_mask=observed_mask & artificial_mask

    #Apply masking
    X_masked=X.copy()
    X_masked[combined_mask]=np.nan
    return X_masked,combined_mask


#Use mcar on validation data 
val_X_masked, val_mask =mcar_f(val_data)
val_X_ori=val_data.copy() #Ground truth

test_X_masked, test_mask =mcar_f(test_data)
test_X_ori=test_data.copy() #ground truth

#?? Problem: Can't have the best input for testing
#1.Create synthetic test_data cuz if I drop nan values for test set, there's basically nothing left
#synthetic_data=np.random.randn(n_samples,n_steps,n_features)
#test_X_masked,test_mask=mcar_f(synthetic_data)
#test_X_ori=synthetic_data.copy() #Ground truth

# 2, Ensure no NaN values in synthetic data
#test_X_masked = np.nan_to_num(test_X_masked, nan=np.nanmean(test_X_masked))
#test_X_ori = np.nan_to_num(test_X_ori, nan=np.nanmean(test_X_ori))


#3. Set aside a part of dataset for test_set
#df_test=df1[6000:8000]  #Even with 10000 rows of dataset fot testing, after dropping nan value in rows, there is none left 
#2. Drop nan rows 
#f_test=df_test.dropna(axis=0,how='all')
#print("Size of the test set:", len(df_test)) 
#data_test=df_test[sensor_cols].values



#n_features = data_test.shape[1]  # exclude the time column
#n_steps = 60 #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
#n_samples = data_test.shape[0] // n_steps 
#Reshape and apply mcar to the test set 
# #data_test=data_test[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)

#data_test_normalized=np.zeros_like(data_test)  # Initialize the normalized data array
#for i in range(data_test.shape[2]):
  # scaler = MinMaxScaler(feature_range=(0, 1))
    # Flatten timesteps and samples for scaling
  # data_test_normalized[:, :, i] = scaler.fit_transform(data_test[:, :, i].reshape(-1, 1)).reshape(data_test.shape[0], data_test.shape[1])
  # scalers[i] = scaler  # Save scalers to inverse-transform later




class Config:
    no_cuda = False
    no_mps = False
    seed = 1

args=Config()

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)


args.cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

if args.cuda:
    device = torch.device("cuda")
    print("Using CUDA")
elif use_mps:
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")


#MLflow set up
mlflow.set_tracking_uri("http://localhost:5000")
client = mlflow.tracking.MlflowClient()
mlflow.set_experiment("SAITS")
SAITS_run_name = "SAITS"

with mlflow.start_run(run_name=SAITS_run_name) as run:
# initialize the model (from example)
    saits = SAITS(
        n_steps=data_normalized.shape[1],
        n_features=data_normalized.shape[2],
        n_layers=2, #deep network(4) for long sequences, here is revised for 2 since it always shows cuda error 
        d_model=512,  #d_modle must equal n_heads * d_k
        d_ffn=512,
        n_heads=8,
        d_k=64,
        d_v=64,
        dropout=0.1,
        attn_dropout=0.1,
        diagonal_attention_mask=True,  # otherwise the original self-attention mechanism will be applied
        ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
       # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
        MIT_weight=1,
        batch_size=5, #try with 5 to see if it works, was 4. 
        # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
        epochs=10, #try with 10 to see if it runs
        # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
        # You can leave it to defualt as None to disable early stopping.
        patience=4, #initially was 3, tried to increase to see more possibilities 
        # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
        # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
        optimizer=Adam(lr=1e-3),
        # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
        # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
        # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
        num_workers=0,
        # just leave it to default as None, PyPOTS will automatically assign the best device for you.
        # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
        device=device, 
        # set the path for saving tensorboard and trained model files 
        saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model",
        # only save the best model after training finished.
        # You can also set it as "better" to save models performing better ever during training.
        model_saving_strategy="best",
)



# Log model parameters
mlflow.log_param("n_steps", data_normalized.shape[1])
mlflow.log_param("n_features", data_normalized.shape[2])
mlflow.log_param("batch_size", 5)
mlflow.log_param("epochs", 10)
mlflow.log_param("patience", 4)


# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
#use original missigness for trainig, use the mcar masked points as ground truth
saits.fit(train_set={"X": train_data}, val_set={"X": val_X_masked , "X_ori":val_X_ori })


##drop null values in test set 

# Check for NaN values across the entire array
#nan_mask = np.isnan(test_X).any(axis=(1, 2))

# Filter out samples (rows) that contain NaN values
#test_X_clean = test_X[~nan_mask]
#test_X_ori = test_X_clean
#test_X_masked, test_mask = mcar_f(test_X_clean)

# the testing stage, impute the originally-missing values and artificially-missing values in the test set
# Convert the numpy array to a dictionary
test_set_dict = {"X": test_X_masked}
test_imputation = saits.predict(test_set_dict)

#change back to array for applying inverse scale later
test_imputation_array = test_imputation["imputation"]


#clear cuda after prediction if on cuda

# Ensure saits_imputation and test_X are numpy arrays
#if not isinstance(saits_imputation, np.ndarray):
    #saits_imputation = np.array(saits_imputation)
#if not isinstance(test_X, np.ndarray):
     #test_X = np.array(test_X)

#Inverse Scale
def inverse_scale(imputation, scalers):
    n_features = imputation.shape[2]
    imputation_denorm = np.empty_like(imputation)
    
    for i in range(n_features):
        imputation_denorm[:, :, i] = scalers[i].inverse_transform(imputation[:, :, i].reshape(-1, 1)).reshape(imputation.shape[0], imputation.shape[1])
    
    return imputation_denorm  # Move the return statement outside the loop

    
#Apply function to the dataset 
test_imputation_denorm=inverse_scale(test_imputation_array,scalers)
test_X_ori_denorm=inverse_scale(test_X_ori,scalers)

if args.cuda:
    torch.cuda.empty_cache() 


#Calculate metrics per-feature: mean absolute error on the ground truth (artificially-missing values)
mae_per_feature=[]
for i in range(n_features):
    #Extract imputation and ground truth for feature i
    imputation_i=test_imputation_denorm[:,:,i]
    ground_truth_i=test_X_ori_denorm[:,:,i]
    mask_i=test_mask[:,:,i]
    # Check for NaN values
    if np.isnan(imputation_i).any() or np.isnan(ground_truth_i).any():
        print(f"NaN values detected in feature {i}")
        continue  # Skip this feature if NaN values are found
    #Filter only artificially masked positions
    mae_i=calc_mae(imputation_i,ground_truth_i,mask_i)
    mae_per_feature.append(mae_i)
    print(f"MAE for {feature_names[i]}: {mae_i:.4f}")
    mlflow.log_metric(f"MAE_{feature_names[i]}", mae_i)


# calculate average MAE 
avg_mae=np.mean(mae_per_feature)
print(f"Testing mean absolute error: {avg_mae:.4f}")
mlflow.log_metric("avg_mae", avg_mae)

mlflow.end_run()

##Current error is on the cuda 
#test memory
#def test_memory(in_size=100, out_size=10,hidden_size=100,optimizer_type=torch.optim.Adam, batch_size=1, device=0):
   # sample_input=torch.randn(batch_size,in_size)
  #  model=saits,
   # optimizer=optimizer_type(model.parameters(),lr=1e-3)
   # print("Beginning mem:",torch.cuda.memory_allocated(device))
   # model.to(device)
   # print()
   # for i in range(3):
   #     print("Iteration",i)
   #     a=torch.cuda.memory_allocated(device)
   #     out=model(sample_input.to(device)).sum()
   #     b=torch.cuda.memory_allocated(device)
   #     print("2-After forward pass", torch.cuda.memory_allocated(device))
   #     print("2-Memory consumed by forward pass", b-a)
   #     out.backward()
   #     print("3-After backward pass", torch.cuda.memory_allocated(device))
   #     optimizer.step()
   #     print("4-After optimizer step", torch.cuda.memory_allocated(device))

##To do: log mlflow, 

Column time has 0 NaN values
Column time has 0.0 Missing_rate
Column fr_eng has 0 NaN values
Column fr_eng has 0.0 Missing_rate
Column te_exh_cyl_out__0 has 73 NaN values
Column te_exh_cyl_out__0 has 0.0006111906496203082 Missing_rate
Column pd_air_ic__0 has 73 NaN values
Column pd_air_ic__0 has 0.0006111906496203082 Missing_rate
Column pr_exh_turb_out__0 has 119439 NaN values
Column pr_exh_turb_out__0 has 1.0 Missing_rate
Column te_air_ic_out__0 has 73 NaN values
Column te_air_ic_out__0 has 0.0006111906496203082 Missing_rate
Column te_seawater has 73 NaN values
Column te_seawater has 0.0006111906496203082 Missing_rate
Column te_air_comp_in_a__0 has 119439 NaN values
Column te_air_comp_in_a__0 has 1.0 Missing_rate
Column te_air_comp_in_b__0 has 119439 NaN values
Column te_air_comp_in_b__0 has 1.0 Missing_rate
Column fr_tc__0 has 119439 NaN values
Column fr_tc__0 has 1.0 Missing_rate
Column pr_baro has 73 NaN values
Column pr_baro has 0.0006111906496203082 Missing_rate
Column pd_air_ic_

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

Exception: Run with UUID a5285036106e4611b162d9ad8f532b5a is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True

In [ ]:
mlflow.end_run()

🏃 View run intelligent-worm-859 at: http://localhost:5000/#/experiments/1/runs/a5285036106e4611b162d9ad8f532b5a
🧪 View experiment at: http://localhost:5000/#/experiments/1


In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import SAITS
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import shap
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
import optuna 
from optuna.visualization import plot_optimization_history



from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/Newdataframeforpypots.parquet")

len(df)

#current length of the dataframe is 119439

# Check nan values in each column
for col in df.columns:
    print(f"Column {col} has {df[col].isna().sum()} NaN values")
    missing_rate=df[col].isna().sum()/len(df[col])
    print(f"Column {col} has {missing_rate} Missing_rate")


#Try with smaller dataset, size 4000
##SAMPLE the percengtage of the dataset, df.sample (averagely pick samples)
#not df.sample cuz it will randomly select 
original_size=len(df)
desired_fraction=0.10
step=int(1/desired_fraction) #step_size=10 (sample every 10th minute)

#Systematic sampling: Start at a random offset to avoid bias 
start=np.random.randint(0,step) #Random start between 0-9
df1=df.iloc[start::step].reset_index(drop=True)

print(f"Original size:{len(df)}, Sampled size: {len(df1)}")


#df1=df[:10000]


# Custom Dataset class
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Data processing code
sensor_cols = [col for col in df1.columns if col != "time"]
data = df1[sensor_cols].values

#¤get feature names for printing mae later 
feature_names=df1[sensor_cols].columns.tolist()

## Convert data to 3D arrays of shape n_samples, n_timesteps, n_features, X_ori refers to the original data without missing values 
## Reconstruct all columns simultaneously  #num_features: 119
n_features = data.shape[1]  # exclude the time column
n_steps = 30 #60 (was 60 previously) #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data.shape[0] // n_steps 



# Reshape to (n_samples // n_steps, n_steps, n_features)
#data_reshaped = data.reshape((n_samples, n_steps, n_features))
data_reshaped=data[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)
print(f"Reshaped data:{data.shape}")



##Normalization is important because of the nature of mse calculation of saits, columns with large 
#values dominate the loss, making metrics meaningless. SAITS computes MSE/MAE column-wise and averages 
#them across all columns 
#  Apply minmax scaler here 
#normalize each feature independently
scalers={}
data_normalized=data.copy()

data_normalized = np.zeros_like(data_reshaped)  # Initialize the normalized data array
scalers = {}

for i in range(data_reshaped.shape[2]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Flatten timesteps and samples for scaling
    data_normalized[:, :, i] = scaler.fit_transform(data_reshaped[:, :, i].reshape(-1, 1)).reshape(data_reshaped.shape[0], data_reshaped.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later



#Split into train, test, val

train_size = int(0.6 * len(data_normalized))
val_size = int(0.2 * len(data_normalized))
test_size = len(data_normalized) - train_size - val_size

train_data = data_normalized[:train_size]
val_data = data_normalized[train_size:train_size + val_size]
test_data= data_normalized[train_size + val_size:]

#Optional: Artificially mask. Mask 20% of the data (MIT part)
def mcar_f(X, mask_ratio=0.2):
    """Apply MCAR only to observed values."""
    observed_mask=~np.isnan(X) #find observed positions
    artificial_mask=mcar(X,mask_ratio).astype(bool) #generate MCAR mask, cast to boolean
    #combine masks 
    combined_mask=observed_mask & artificial_mask

    #Apply masking
    X_masked=X.copy()
    X_masked[combined_mask]=np.nan
    return X_masked,combined_mask


#Use mcar on validation data 
val_X_masked, val_mask =mcar_f(val_data)
val_X_ori=val_data.copy() #Ground truth

test_X_masked, test_mask =mcar_f(test_data)
test_X_ori=test_data.copy() #ground truth

#?? Problem: Can't have the best input for testing
#1.Create synthetic test_data cuz if I drop nan values for test set, there's basically nothing left
#synthetic_data=np.random.randn(n_samples,n_steps,n_features)
#test_X_masked,test_mask=mcar_f(synthetic_data)
#test_X_ori=synthetic_data.copy() #Ground truth

# 2, Ensure no NaN values in synthetic data
#test_X_masked = np.nan_to_num(test_X_masked, nan=np.nanmean(test_X_masked))
#test_X_ori = np.nan_to_num(test_X_ori, nan=np.nanmean(test_X_ori))


#3. Set aside a part of dataset for test_set
#df_test=df1[6000:8000]  #Even with 10000 rows of dataset fot testing, after dropping nan value in rows, there is none left 
#2. Drop nan rows 
#f_test=df_test.dropna(axis=0,how='all')
#print("Size of the test set:", len(df_test)) 
#data_test=df_test[sensor_cols].values



#n_features = data_test.shape[1]  # exclude the time column
#n_steps = 60 #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
#n_samples = data_test.shape[0] // n_steps 
#Reshape and apply mcar to the test set 
# #data_test=data_test[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)

#data_test_normalized=np.zeros_like(data_test)  # Initialize the normalized data array
#for i in range(data_test.shape[2]):
  # scaler = MinMaxScaler(feature_range=(0, 1))
    # Flatten timesteps and samples for scaling
  # data_test_normalized[:, :, i] = scaler.fit_transform(data_test[:, :, i].reshape(-1, 1)).reshape(data_test.shape[0], data_test.shape[1])
  # scalers[i] = scaler  # Save scalers to inverse-transform later




class Config:
    no_cuda = False
    no_mps = False
    seed = 1

args=Config()

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)


args.cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

if args.cuda:
    device = torch.device("cuda")
    print("Using CUDA")
elif use_mps:
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")


#MLflow set up
mlflow.set_tracking_uri("http://localhost:5000")
client = mlflow.tracking.MlflowClient()
mlflow.set_experiment("SAITS")
SAITS_run_name = "SAITS"

with mlflow.start_run(run_name=SAITS_run_name) as run:
# initialize the model (from example)
    saits = SAITS(
        n_steps=data_normalized.shape[1],
        n_features=data_normalized.shape[2],
        n_layers=2, #deep network(4) for long sequences, here is revised for 2 since it always shows cuda error 
        d_model=512,  #d_modle must equal n_heads * d_k
        d_ffn=512,
        n_heads=8,
        d_k=64,
        d_v=64,
        dropout=0.1,
        attn_dropout=0.1,
        diagonal_attention_mask=True,  # otherwise the original self-attention mechanism will be applied
        ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
       # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
        MIT_weight=1,
        batch_size=5, #try with 5 to see if it works, was 4. 
        # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
        epochs=10, #try with 10 to see if it runs
        # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
        # You can leave it to defualt as None to disable early stopping.
        patience=4, #initially was 3, tried to increase to see more possibilities 
        # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
        # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
        optimizer=Adam(lr=1e-3),
        # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
        # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
        # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
        num_workers=0,
        # just leave it to default as None, PyPOTS will automatically assign the best device for you.
        # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
        device=device, 
        # set the path for saving tensorboard and trained model files 
        saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model",
        # only save the best model after training finished.
        # You can also set it as "better" to save models performing better ever during training.
        model_saving_strategy="best",
)



# Log model parameters
mlflow.log_param("n_steps", data_normalized.shape[1])
mlflow.log_param("n_features", data_normalized.shape[2])
mlflow.log_param("batch_size", 5)
mlflow.log_param("epochs", 10)
mlflow.log_param("patience", 4)


# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
#use original missigness for trainig, use the mcar masked points as ground truth
saits.fit(train_set={"X": train_data}, val_set={"X": val_X_masked , "X_ori":val_X_ori })


##drop null values in test set 

# Check for NaN values across the entire array
#nan_mask = np.isnan(test_X).any(axis=(1, 2))

# Filter out samples (rows) that contain NaN values
#test_X_clean = test_X[~nan_mask]
#test_X_ori = test_X_clean
#test_X_masked, test_mask = mcar_f(test_X_clean)

# the testing stage, impute the originally-missing values and artificially-missing values in the test set
# Convert the numpy array to a dictionary
test_set_dict = {"X": test_X_masked}
test_imputation = saits.predict(test_set_dict)

#change back to array for applying inverse scale later
test_imputation_array = test_imputation["imputation"]


#clear cuda after prediction if on cuda

# Ensure saits_imputation and test_X are numpy arrays
#if not isinstance(saits_imputation, np.ndarray):
    #saits_imputation = np.array(saits_imputation)
#if not isinstance(test_X, np.ndarray):
     #test_X = np.array(test_X)

#Inverse Scale
def inverse_scale(imputation, scalers):
    n_features = imputation.shape[2]
    imputation_denorm = np.empty_like(imputation)
    
    for i in range(n_features):
        imputation_denorm[:, :, i] = scalers[i].inverse_transform(imputation[:, :, i].reshape(-1, 1)).reshape(imputation.shape[0], imputation.shape[1])
    
    return imputation_denorm  # Move the return statement outside the loop

    
#Apply function to the dataset 
test_imputation_denorm=inverse_scale(test_imputation_array,scalers)
test_X_ori_denorm=inverse_scale(test_X_ori,scalers)

if args.cuda:
    torch.cuda.empty_cache() 


#Calculate metrics per-feature: mean absolute error on the ground truth (artificially-missing values)
mae_per_feature=[]
for i in range(n_features):
    #Extract imputation and ground truth for feature i
    imputation_i=test_imputation_denorm[:,:,i]
    ground_truth_i=test_X_ori_denorm[:,:,i]
    mask_i=test_mask[:,:,i]
    # Check for NaN values
    if np.isnan(imputation_i).any() or np.isnan(ground_truth_i).any():
        print(f"NaN values detected in feature {i}")
        continue  # Skip this feature if NaN values are found
    #Filter only artificially masked positions
    mae_i=calc_mae(imputation_i,ground_truth_i,mask_i)
    mae_per_feature.append(mae_i)
    print(f"MAE for {feature_names[i]}: {mae_i:.4f}")
    mlflow.log_metric(f"MAE_{feature_names[i]}", mae_i)


# calculate average MAE 
avg_mae=np.mean(mae_per_feature)
print(f"Testing mean absolute error: {avg_mae:.4f}")
mlflow.log_metric("avg_mae", avg_mae)

mlflow.end_run()

##Current error is on the cuda 
#test memory
#def test_memory(in_size=100, out_size=10,hidden_size=100,optimizer_type=torch.optim.Adam, batch_size=1, device=0):
   # sample_input=torch.randn(batch_size,in_size)
  #  model=saits,
   # optimizer=optimizer_type(model.parameters(),lr=1e-3)
   # print("Beginning mem:",torch.cuda.memory_allocated(device))
   # model.to(device)
   # print()
   # for i in range(3):
   #     print("Iteration",i)
   #     a=torch.cuda.memory_allocated(device)
   #     out=model(sample_input.to(device)).sum()
   #     b=torch.cuda.memory_allocated(device)
   #     print("2-After forward pass", torch.cuda.memory_allocated(device))
   #     print("2-Memory consumed by forward pass", b-a)
   #     out.backward()
   #     print("3-After backward pass", torch.cuda.memory_allocated(device))
   #     optimizer.step()
   #     print("4-After optimizer step", torch.cuda.memory_allocated(device))

##To do: log mlflow, 

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

Column time has 0 NaN values
Column time has 0.0 Missing_rate
Column fr_eng has 0 NaN values
Column fr_eng has 0.0 Missing_rate
Column te_exh_cyl_out__0 has 73 NaN values
Column te_exh_cyl_out__0 has 0.0006111906496203082 Missing_rate
Column pd_air_ic__0 has 73 NaN values
Column pd_air_ic__0 has 0.0006111906496203082 Missing_rate
Column pr_exh_turb_out__0 has 119439 NaN values
Column pr_exh_turb_out__0 has 1.0 Missing_rate
Column te_air_ic_out__0 has 73 NaN values
Column te_air_ic_out__0 has 0.0006111906496203082 Missing_rate
Column te_seawater has 73 NaN values
Column te_seawater has 0.0006111906496203082 Missing_rate
Column te_air_comp_in_a__0 has 119439 NaN values
Column te_air_comp_in_a__0 has 1.0 Missing_rate
Column te_air_comp_in_b__0 has 119439 NaN values
Column te_air_comp_in_b__0 has 1.0 Missing_rate
Column fr_tc__0 has 119439 NaN values
Column fr_tc__0 has 1.0 Missing_rate
Column pr_baro has 73 NaN values
Column pr_baro has 0.0006111906496203082 Missing_rate
Column pd_air_ic_

2025-05-13 21:45:25 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 6,402,976


🏃 View run SAITS at: http://localhost:5000/#/experiments/1/runs/91558134e43f4cb691a4e2c055ce0621
🧪 View experiment at: http://localhost:5000/#/experiments/1


2025-05-13 21:45:32 [INFO]: Epoch 001 - training loss (MAE): 0.4964, validation MSE: 0.0687
2025-05-13 21:45:39 [INFO]: Epoch 002 - training loss (MAE): 0.2534, validation MSE: 0.0921
2025-05-13 21:45:45 [INFO]: Epoch 003 - training loss (MAE): 0.2250, validation MSE: 0.1182
2025-05-13 21:45:52 [INFO]: Epoch 004 - training loss (MAE): 0.1784, validation MSE: 0.0729
2025-05-13 21:45:59 [INFO]: Epoch 005 - training loss (MAE): 0.1823, validation MSE: 0.0942
2025-05-13 21:45:59 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-05-13 21:45:59 [INFO]: Finished training. The best model is from epoch#1.
2025-05-13 21:45:59 [INFO]: Saved the model to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model/20250513_T214525/SAITS.pypots


MAE for fr_eng: 0.1380
NaN values detected in feature 1
NaN values detected in feature 2
NaN values detected in feature 3
NaN values detected in feature 4
NaN values detected in feature 5
NaN values detected in feature 6
NaN values detected in feature 7
NaN values detected in feature 8
NaN values detected in feature 9
NaN values detected in feature 10
NaN values detected in feature 11
NaN values detected in feature 12
NaN values detected in feature 13
MAE for bo_aux_blower_running: 0.4142
NaN values detected in feature 15
MAE for pr_air_scav_ecs: 60613.2628
MAE for pr_air_scav: 35925.2965
NaN values detected in feature 18
NaN values detected in feature 19
NaN values detected in feature 20
NaN values detected in feature 21
NaN values detected in feature 22
NaN values detected in feature 23
MAE for fr_eng_setpoint: 0.2241
NaN values detected in feature 25
NaN values detected in feature 26
NaN values detected in feature 27
MAE for fr_eng_ecs: 0.1385
NaN values detected in feature 29
MAE f

In [ ]:
df1["pr_air_scav_ecs"].std()

64122.002728967054

In [ ]:
df1["te_air_scav_rec"].std()

3.0230871705516225

In [ ]:
df1["fr_eng_ecs"].std()

0.2105815625902985

In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import SAITS
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import shap
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
import optuna 
from optuna.visualization import plot_optimization_history



from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/Newdataframeforpypots.parquet")

len(df)

#current length of the dataframe is 119439

# Check nan values in each column
for col in df.columns:
    print(f"Column {col} has {df[col].isna().sum()} NaN values")
    missing_rate=df[col].isna().sum()/len(df[col])
    print(f"Column {col} has {missing_rate} Missing_rate")


#Try with smaller dataset, size 4000
##SAMPLE the percengtage of the dataset, df.sample (averagely pick samples)
#not df.sample cuz it will randomly select 
original_size=len(df)
desired_fraction=0.3
step=int(1/desired_fraction) #step_size=10 (sample every 10th (3/10) minute)

#Systematic sampling: Start at a random offset to avoid bias 
start=np.random.randint(0,step) #Random start between 0-9
df1=df.iloc[start::step].reset_index(drop=True)

print(f"Original size:{len(df)}, Sampled size: {len(df1)}")


#df1=df[:10000]


# Custom Dataset class
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Data processing code
sensor_cols = [col for col in df1.columns if col != "time"]
data = df1[sensor_cols].values

#¤get feature names for printing mae later 
feature_names=df1[sensor_cols].columns.tolist()

## Convert data to 3D arrays of shape n_samples, n_timesteps, n_features, X_ori refers to the original data without missing values 
## Reconstruct all columns simultaneously  #num_features: 119
n_features = data.shape[1]  # exclude the time column
n_steps = 30 #60 (was 60 previously) #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data.shape[0] // n_steps 



# Reshape to (n_samples // n_steps, n_steps, n_features)
#data_reshaped = data.reshape((n_samples, n_steps, n_features))
data_reshaped=data[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)
print(f"Reshaped data:{data.shape}")



##Normalization is important because of the nature of mse calculation of saits, columns with large 
#values dominate the loss, making metrics meaningless. SAITS computes MSE/MAE column-wise and averages 
#them across all columns 
#  Apply minmax scaler here 
#normalize each feature independently
scalers={}
data_normalized=data.copy()

data_normalized = np.zeros_like(data_reshaped)  # Initialize the normalized data array
scalers = {}

for i in range(data_reshaped.shape[2]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Flatten timesteps and samples for scaling
    data_normalized[:, :, i] = scaler.fit_transform(data_reshaped[:, :, i].reshape(-1, 1)).reshape(data_reshaped.shape[0], data_reshaped.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later



#Split into train, test, val

train_size = int(0.6 * len(data_normalized))
val_size = int(0.2 * len(data_normalized))
test_size = len(data_normalized) - train_size - val_size

train_data = data_normalized[:train_size]
val_data = data_normalized[train_size:train_size + val_size]
test_data= data_normalized[train_size + val_size:]

#Optional: Artificially mask. Mask 20% of the data (MIT part)
def mcar_f(X, mask_ratio=0.2):
    """Apply MCAR only to observed values."""
    observed_mask=~np.isnan(X) #find observed positions
    artificial_mask=mcar(X,mask_ratio).astype(bool) #generate MCAR mask, cast to boolean
    #combine masks 
    combined_mask=observed_mask & artificial_mask

    #Apply masking
    X_masked=X.copy()
    X_masked[combined_mask]=np.nan
    return X_masked,combined_mask


#Use mcar on validation data 
val_X_masked, val_mask =mcar_f(val_data)
val_X_ori=val_data.copy() #Ground truth

test_X_masked, test_mask =mcar_f(test_data)
test_X_ori=test_data.copy() #ground truth

#?? Problem: Can't have the best input for testing
#1.Create synthetic test_data cuz if I drop nan values for test set, there's basically nothing left
#synthetic_data=np.random.randn(n_samples,n_steps,n_features)
#test_X_masked,test_mask=mcar_f(synthetic_data)
#test_X_ori=synthetic_data.copy() #Ground truth

# 2, Ensure no NaN values in synthetic data
#test_X_masked = np.nan_to_num(test_X_masked, nan=np.nanmean(test_X_masked))
#test_X_ori = np.nan_to_num(test_X_ori, nan=np.nanmean(test_X_ori))


#3. Set aside a part of dataset for test_set
#df_test=df1[6000:8000]  #Even with 10000 rows of dataset fot testing, after dropping nan value in rows, there is none left 
#2. Drop nan rows 
#f_test=df_test.dropna(axis=0,how='all')
#print("Size of the test set:", len(df_test)) 
#data_test=df_test[sensor_cols].values



#n_features = data_test.shape[1]  # exclude the time column
#n_steps = 60 #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
#n_samples = data_test.shape[0] // n_steps 
#Reshape and apply mcar to the test set 
# #data_test=data_test[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)

#data_test_normalized=np.zeros_like(data_test)  # Initialize the normalized data array
#for i in range(data_test.shape[2]):
  # scaler = MinMaxScaler(feature_range=(0, 1))
    # Flatten timesteps and samples for scaling
  # data_test_normalized[:, :, i] = scaler.fit_transform(data_test[:, :, i].reshape(-1, 1)).reshape(data_test.shape[0], data_test.shape[1])
  # scalers[i] = scaler  # Save scalers to inverse-transform later




class Config:
    no_cuda = False
    no_mps = False
    seed = 1

args=Config()

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)


args.cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

if args.cuda:
    device = torch.device("cuda")
    print("Using CUDA")
elif use_mps:
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")


#MLflow set up
mlflow.set_tracking_uri("http://localhost:5000")
client = mlflow.tracking.MlflowClient()
mlflow.set_experiment("SAITS")
SAITS_run_name = "SAITS"

with mlflow.start_run(run_name=SAITS_run_name) as run:
# initialize the model (from example)
    saits = SAITS(
        n_steps=data_normalized.shape[1],
        n_features=data_normalized.shape[2],
        n_layers=2, #deep network(4) for long sequences, here is revised for 2 since it always shows cuda error 
        d_model=512,  #d_modle must equal n_heads * d_k
        d_ffn=512,
        n_heads=8,
        d_k=64,
        d_v=64,
        dropout=0.1,
        attn_dropout=0.1,
        diagonal_attention_mask=True,  # otherwise the original self-attention mechanism will be applied
        ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
       # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
        MIT_weight=1,
        batch_size=5, #try with 5 to see if it works, was 4. 
        # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
        epochs=10, #try with 10 to see if it runs
        # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
        # You can leave it to defualt as None to disable early stopping.
        patience=6, #initially was 3, tried to increase to see more possibilities 
        # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
        # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
        optimizer=Adam(lr=1e-2),
        # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
        # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
        # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
        num_workers=0,
        # just leave it to default as None, PyPOTS will automatically assign the best device for you.
        # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
        device=device, 
        # set the path for saving tensorboard and trained model files 
        saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model",
        # only save the best model after training finished.
        # You can also set it as "better" to save models performing better ever during training.
        model_saving_strategy="best",
)



# Log model parameters
mlflow.log_param("n_steps", data_normalized.shape[1])
mlflow.log_param("n_features", data_normalized.shape[2])
mlflow.log_param("batch_size", 5)
mlflow.log_param("epochs", 10)
mlflow.log_param("patience", 4)


# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
#use original missigness for trainig, use the mcar masked points as ground truth
saits.fit(train_set={"X": train_data}, val_set={"X": val_X_masked , "X_ori":val_X_ori })


##drop null values in test set 

# Check for NaN values across the entire array
#nan_mask = np.isnan(test_X).any(axis=(1, 2))

# Filter out samples (rows) that contain NaN values
#test_X_clean = test_X[~nan_mask]
#test_X_ori = test_X_clean
#test_X_masked, test_mask = mcar_f(test_X_clean)

# the testing stage, impute the originally-missing values and artificially-missing values in the test set
# Convert the numpy array to a dictionary
test_set_dict = {"X": test_X_masked}
test_imputation = saits.predict(test_set_dict)

#change back to array for applying inverse scale later
test_imputation_array = test_imputation["imputation"]


#clear cuda after prediction if on cuda

# Ensure saits_imputation and test_X are numpy arrays
#if not isinstance(saits_imputation, np.ndarray):
    #saits_imputation = np.array(saits_imputation)
#if not isinstance(test_X, np.ndarray):
     #test_X = np.array(test_X)

#Inverse Scale
def inverse_scale(imputation, scalers):
    n_features = imputation.shape[2]
    imputation_denorm = np.empty_like(imputation)
    
    for i in range(n_features):
        imputation_denorm[:, :, i] = scalers[i].inverse_transform(imputation[:, :, i].reshape(-1, 1)).reshape(imputation.shape[0], imputation.shape[1])
    
    return imputation_denorm  # Move the return statement outside the loop

    
#Apply function to the dataset 
test_imputation_denorm=inverse_scale(test_imputation_array,scalers)
test_X_ori_denorm=inverse_scale(test_X_ori,scalers)

if args.cuda:
    torch.cuda.empty_cache() 


#Calculate metrics per-feature: mean absolute error on the ground truth (artificially-missing values)
mae_per_feature=[]
for i in range(n_features):
    #Extract imputation and ground truth for feature i
    imputation_i=test_imputation_denorm[:,:,i]
    ground_truth_i=test_X_ori_denorm[:,:,i]
    mask_i=test_mask[:,:,i]
    # Check for NaN values
    if np.isnan(imputation_i).any() or np.isnan(ground_truth_i).any():
        print(f"NaN values detected in feature {i}")
        continue  # Skip this feature if NaN values are found
    #Filter only artificially masked positions
    mae_i=calc_mae(imputation_i,ground_truth_i,mask_i)
    mae_per_feature.append(mae_i)
    print(f"MAE for {feature_names[i]}: {mae_i:.4f}")
    mlflow.log_metric(f"MAE_{feature_names[i]}", mae_i)


# calculate average MAE 
avg_mae=np.mean(mae_per_feature)
print(f"Testing mean absolute error: {avg_mae:.4f}")
mlflow.log_metric("avg_mae", avg_mae)

mlflow.end_run()

##Current error is on the cuda 
#test memory
#def test_memory(in_size=100, out_size=10,hidden_size=100,optimizer_type=torch.optim.Adam, batch_size=1, device=0):
   # sample_input=torch.randn(batch_size,in_size)
  #  model=saits,
   # optimizer=optimizer_type(model.parameters(),lr=1e-3)
   # print("Beginning mem:",torch.cuda.memory_allocated(device))
   # model.to(device)
   # print()
   # for i in range(3):
   #     print("Iteration",i)
   #     a=torch.cuda.memory_allocated(device)
   #     out=model(sample_input.to(device)).sum()
   #     b=torch.cuda.memory_allocated(device)
   #     print("2-After forward pass", torch.cuda.memory_allocated(device))
   #     print("2-Memory consumed by forward pass", b-a)
   #     out.backward()
   #     print("3-After backward pass", torch.cuda.memory_allocated(device))
   #     optimizer.step()
   #     print("4-After optimizer step", torch.cuda.memory_allocated(device))

##To do: log mlflow, 

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

Column time has 0 NaN values
Column time has 0.0 Missing_rate
Column fr_eng has 0 NaN values
Column fr_eng has 0.0 Missing_rate
Column te_exh_cyl_out__0 has 73 NaN values
Column te_exh_cyl_out__0 has 0.0006111906496203082 Missing_rate
Column pd_air_ic__0 has 73 NaN values
Column pd_air_ic__0 has 0.0006111906496203082 Missing_rate
Column pr_exh_turb_out__0 has 119439 NaN values
Column pr_exh_turb_out__0 has 1.0 Missing_rate
Column te_air_ic_out__0 has 73 NaN values
Column te_air_ic_out__0 has 0.0006111906496203082 Missing_rate
Column te_seawater has 73 NaN values
Column te_seawater has 0.0006111906496203082 Missing_rate
Column te_air_comp_in_a__0 has 119439 NaN values
Column te_air_comp_in_a__0 has 1.0 Missing_rate
Column te_air_comp_in_b__0 has 119439 NaN values
Column te_air_comp_in_b__0 has 1.0 Missing_rate
Column fr_tc__0 has 119439 NaN values
Column fr_tc__0 has 1.0 Missing_rate
Column pr_baro has 73 NaN values
Column pr_baro has 0.0006111906496203082 Missing_rate
Column pd_air_ic_

2025-05-13 22:35:21 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 6,402,976


🏃 View run SAITS at: http://localhost:5000/#/experiments/1/runs/f1af00359cf949f5ae1d71fc286cc84d
🧪 View experiment at: http://localhost:5000/#/experiments/1


2025-05-13 22:36:06 [INFO]: Epoch 001 - training loss (MAE): 0.5610, validation MSE: 0.0698
2025-05-13 22:36:53 [INFO]: Epoch 002 - training loss (MAE): 0.3486, validation MSE: 0.0634
2025-05-13 22:37:42 [INFO]: Epoch 003 - training loss (MAE): 0.3429, validation MSE: 0.0599
2025-05-13 22:38:30 [INFO]: Epoch 004 - training loss (MAE): 0.3448, validation MSE: 0.0667
2025-05-13 22:39:25 [INFO]: Epoch 005 - training loss (MAE): 0.3420, validation MSE: 0.0675
2025-05-13 22:40:22 [INFO]: Epoch 006 - training loss (MAE): 0.3421, validation MSE: 0.0633
2025-05-13 22:41:17 [INFO]: Epoch 007 - training loss (MAE): 0.3421, validation MSE: 0.0640
2025-05-13 22:42:14 [INFO]: Epoch 008 - training loss (MAE): 0.3426, validation MSE: 0.0634
2025-05-13 22:43:15 [INFO]: Epoch 009 - training loss (MAE): 0.3398, validation MSE: 0.0681
2025-05-13 22:43:15 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-05-13 22:43:15 [INFO]: Finished training. The best model is from epoc

MAE for fr_eng: 0.1329
NaN values detected in feature 1
NaN values detected in feature 2
NaN values detected in feature 3
NaN values detected in feature 4
NaN values detected in feature 5
NaN values detected in feature 6
NaN values detected in feature 7
NaN values detected in feature 8
NaN values detected in feature 9
NaN values detected in feature 10
NaN values detected in feature 11
NaN values detected in feature 12
NaN values detected in feature 13
MAE for bo_aux_blower_running: 0.5469
NaN values detected in feature 15
MAE for pr_air_scav_ecs: 36343.4720
MAE for pr_air_scav: 36151.9926
NaN values detected in feature 18
NaN values detected in feature 19
NaN values detected in feature 20
NaN values detected in feature 21
NaN values detected in feature 22
NaN values detected in feature 23
MAE for fr_eng_setpoint: 0.1322
NaN values detected in feature 25
NaN values detected in feature 26
NaN values detected in feature 27
MAE for fr_eng_ecs: 0.1466
NaN values detected in feature 29
MAE f

In [ ]:
print(df1)

                     time    fr_eng  te_exh_cyl_out__0  pd_air_ic__0  \
0     2023-10-01 00:01:00  0.732878             510.15         420.0   
1     2023-10-01 00:04:00  0.732730             510.15         430.0   
2     2023-10-01 00:07:00  0.734522             510.15         460.0   
3     2023-10-01 00:10:00  0.733247             510.15         430.0   
4     2023-10-01 00:13:00  0.733089             510.15         430.0   
...                   ...       ...                ...           ...   
39808 2024-01-31 06:16:00  0.350018             406.15         280.0   
39809 2024-01-31 06:19:00  0.350902             405.15         300.0   
39810 2024-01-31 06:22:00  0.350278             402.15         300.0   
39811 2024-01-31 06:25:00  0.348935             401.15         280.0   
39812 2024-01-31 06:28:00  0.186159             399.15         300.0   

       pr_exh_turb_out__0  te_air_ic_out__0  te_seawater  te_air_comp_in_a__0  \
0                     NaN            308.95       273.

In [ ]:
import numpy as np
import mlflow

# Function to calculate MAE while ignoring NaNs
def calc_mae_ignore_nan(imputation, ground_truth, mask):
    # Apply mask to ignore NaNs
    valid_mask = ~np.isnan(imputation) & ~np.isnan(ground_truth) & mask
    mae = np.abs(imputation[valid_mask] - ground_truth[valid_mask]).mean()
    return mae

# Calculate metrics per-feature: mean absolute error on the ground truth (artificially-missing values)
mae_per_feature = []
for i in range(n_features):
    # Extract imputation and ground truth for feature i
    imputation_i = test_imputation_denorm[:, :, i]
    ground_truth_i = test_X_ori_denorm[:, :, i]
    mask_i = test_mask[:, :, i]
    
    # Calculate MAE while ignoring NaNs
    mae_i = calc_mae_ignore_nan(imputation_i, ground_truth_i, mask_i)
    mae_per_feature.append(mae_i)
    print(f"MAE for {feature_names[i]}: {mae_i:.4f}")
    mlflow.log_metric(f"MAE_{feature_names[i]}", mae_i)

# Calculate average MAE
avg_mae = np.mean(mae_per_feature)
print(f"Testing mean absolute error: {avg_mae:.4f}")
mlflow.log_metric("avg_mae", avg_mae)


MAE for fr_eng: 0.1329
MAE for te_exh_cyl_out__0: 20.2169
MAE for pd_air_ic__0: 260.4531
MAE for pr_exh_turb_out__0: nan
MAE for te_air_ic_out__0: 1.9398
MAE for te_seawater: 0.0001
MAE for te_air_comp_in_a__0: nan


<ipython-input-18-6a8f46eb636a>:8: RuntimeWarning: Mean of empty slice.
  mae = np.abs(imputation[valid_mask] - ground_truth[valid_mask]).mean()
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


MAE for te_air_comp_in_b__0: nan
MAE for fr_tc__0: nan
MAE for pr_baro: 826.0051
MAE for pd_air_ic__0_1: 267.2695
MAE for pr_exh_rec: 32717.8773
MAE for te_exh_turb_in__0: nan
MAE for te_exh_turb_out__0: nan
MAE for bo_aux_blower_running: 0.5469
MAE for re_eng_load: 0.1117
MAE for pr_air_scav_ecs: 36343.4720
MAE for pr_air_scav: 36151.9926
MAE for te_air_scav_rec: 4.0500
MAE for te_air_ic_out__0_1: 1.9919
MAE for pr_cyl_comp__0: 2055017.6472
MAE for pr_cyl_max__0: 2345898.8432
MAE for se_mip__0: 179047.5526
MAE for te_exh_cyl_out__0_1: 18.8740
MAE for fr_eng_setpoint: 0.1322
MAE for te_air_scav_rec_iso: 2.1861
MAE for pr_cyl_max_mv_iso: 2484755.9879
MAE for pr_cyl_comp_mv_iso: 2116437.2242
MAE for fr_eng_ecs: 0.1466
MAE for pr_air_scav_iso: 28296.0174
MAE for engine_type_G95ME-C10.5-GI-EGRTC: 0.0034
Testing mean absolute error: nan


In [ ]:
df1["te_air_scav_rec"].std()

3.023644989747263

In [ ]:
df1["pr_baro"].std()

902.3217968207963

In [ ]:


# Function to calculate MAE while ignoring NaNs
def calc_mae_ignore_nan(imputation, ground_truth, mask):
    # Apply mask to ignore NaNs
    valid_mask = ~np.isnan(imputation) & ~np.isnan(ground_truth) & mask
    mae = np.abs(imputation[valid_mask] - ground_truth[valid_mask]).mean()
    return mae

# Calculate metrics per-feature: mean absolute error on the ground truth (artificially-missing values)
mae_per_feature = []
performance_ratios = []
for i in range(n_features):
    # Extract imputation and ground truth for feature i
    imputation_i = test_imputation_denorm[:, :, i]
    ground_truth_i = test_X_ori_denorm[:, :, i]
    mask_i = test_mask[:, :, i]
    
    # Calculate MAE while ignoring NaNs
    mae_i = calc_mae_ignore_nan(imputation_i, ground_truth_i, mask_i)
    mae_per_feature.append(mae_i)
    
    # Calculate standard deviation of the original feature
    std_dev_i = np.nanstd(ground_truth_i)
    
    # Calculate performance ratio (MAE / Standard Deviation)
    performance_ratio_i = mae_i / std_dev_i
    performance_ratios.append(performance_ratio_i)
    
    print(f"MAE for {feature_names[i]}: {mae_i:.4f}")
    print(f"Performance Ratio for {feature_names[i]}: {performance_ratio_i:.4f}")
    
    mlflow.log_metric(f"MAE_{feature_names[i]}", mae_i)
    mlflow.log_metric(f"Performance_Ratio_{feature_names[i]}", performance_ratio_i)

# Calculate average MAE and average performance ratio
avg_mae = np.mean(mae_per_feature)
avg_performance_ratio = np.mean(performance_ratios)

print(f"Testing mean absolute error: {avg_mae:.4f}")
print(f"Average Performance Ratio: {avg_performance_ratio:.4f}")

mlflow.log_metric("avg_mae", avg_mae)
mlflow.log_metric("avg_performance_ratio", avg_performance_ratio)


MAE for fr_eng: 0.1329
Performance Ratio for fr_eng: 0.7238
MAE for te_exh_cyl_out__0: 20.2169
Performance Ratio for te_exh_cyl_out__0: 0.6155
MAE for pd_air_ic__0: 260.4531
Performance Ratio for pd_air_ic__0: 0.9337
MAE for pr_exh_turb_out__0: nan
Performance Ratio for pr_exh_turb_out__0: nan
MAE for te_air_ic_out__0: 1.9398
Performance Ratio for te_air_ic_out__0: 0.7705


<ipython-input-21-528c8751e9ad>:5: RuntimeWarning: Mean of empty slice.
  mae = np.abs(imputation[valid_mask] - ground_truth[valid_mask]).mean()
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
<ipython-input-21-528c8751e9ad>:25: RuntimeWarning: divide by zero encountered in scalar divide
  performance_ratio_i = mae_i / std_dev_i


MAE for te_seawater: 0.0001
Performance Ratio for te_seawater: inf
MAE for te_air_comp_in_a__0: nan
Performance Ratio for te_air_comp_in_a__0: nan
MAE for te_air_comp_in_b__0: nan
Performance Ratio for te_air_comp_in_b__0: nan
MAE for fr_tc__0: nan
Performance Ratio for fr_tc__0: nan
MAE for pr_baro: 826.0051
Performance Ratio for pr_baro: 1.1394
MAE for pd_air_ic__0_1: 267.2695
Performance Ratio for pd_air_ic__0_1: 0.9582
MAE for pr_exh_rec: 32717.8773
Performance Ratio for pr_exh_rec: 0.7650
MAE for te_exh_turb_in__0: nan
Performance Ratio for te_exh_turb_in__0: nan
MAE for te_exh_turb_out__0: nan
Performance Ratio for te_exh_turb_out__0: nan
MAE for bo_aux_blower_running: 0.5469
Performance Ratio for bo_aux_blower_running: 1.2748
MAE for re_eng_load: 0.1117
Performance Ratio for re_eng_load: 0.7356
MAE for pr_air_scav_ecs: 36343.4720
Performance Ratio for pr_air_scav_ecs: 0.7623
MAE for pr_air_scav: 36151.9926
Performance Ratio for pr_air_scav: 0.7582
MAE for te_air_scav_rec: 4.0500

In [ ]:
#Try hyperparameter tuning 

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_layers': [2, 3, 4],
    'd_model': [256, 512, 1024],
    'dropout': [0.1, 0.2, 0.3],
    'learning_rate': [1e-4, 1e-3, 1e-2]
}

# Initialize the model
saits = SAITS(
    n_steps=data_normalized.shape[1],
    n_features=data_normalized.shape[2],
    n_layers=2,
    d_model=512,
    d_ffn=512,
    n_heads=8,
    d_k=64,
    d_v=64,
    dropout=0.1,
    attn_dropout=0.1,
    diagonal_attention_mask=True,
    ORT_weight=1,
    MIT_weight=1,
    batch_size=5,
    epochs=10, 
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=device, 
    saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model",
    model_saving_strategy="best",


    
)

# Perform grid search
grid_search = GridSearchCV(estimator=saits, param_grid=param_grid, cv=3)
grid_search.fit(train_data, val_data)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")


2025-05-13 23:23:00 [INFO]: Using the given device: cpu
2025-05-13 23:23:00 [INFO]: Model files will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model/20250513_T232300
2025-05-13 23:23:00 [INFO]: Tensorboard file will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model/20250513_T232300/tensorboard
2025-05-13 23:23:00 [INFO]: Using customized MAE as the training loss function.
2025-05-13 23:23:00 [INFO]: Using customized MSE as the validation metric function.
2025-05-13 23:23:00 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 6,402,976


TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator <pypots.imputation.saits.model.SAITS object at 0x7f19f1ee8130> does not.

In [ ]:
# %%
from sklearn.model_selection import GridSearchCV


# Define the parameter grid
param_grid = {
    'n_layers': [2, 3, 4],
    'd_model': [256, 512, 1024],
    'dropout': [0.1, 0.2, 0.3],
    'learning_rate': [1e-4, 1e-3, 1e-2]
}

# Initialize the model
saits = SAITS(
    n_steps=data_normalized.shape[1],
    n_features=data_normalized.shape[2],
    n_layers=2,
    d_model=512,
    d_ffn=512,
    n_heads=8,
    d_k=64,
    d_v=64,
    dropout=0.1,
    attn_dropout=0.1,
    diagonal_attention_mask=True,
    ORT_weight=1,
    MIT_weight=1,
    batch_size=5,
    epochs=10, 
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=device, 
    saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model",
    model_saving_strategy="best",


    
)

# Perform grid search
grid_search = GridSearchCV(estimator=saits, param_grid=param_grid, cv=3)
grid_search.fit(train_data, val_data)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")


2025-05-13 23:24:33 [INFO]: Using the given device: cpu
2025-05-13 23:24:33 [INFO]: Model files will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model/20250513_T232433
2025-05-13 23:24:33 [INFO]: Tensorboard file will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model/20250513_T232433/tensorboard
2025-05-13 23:24:33 [INFO]: Using customized MAE as the training loss function.
2025-05-13 23:24:33 [INFO]: Using customized MSE as the validation metric function.
2025-05-13 23:24:33 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 6,402,976


TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator <pypots.imputation.saits.model.SAITS object at 0x7f19f1ee86d0> does not.

In [ ]:
# %%
# %%
from sklearn.model_selection import GridSearchCV


# Custom scoring function
def custom_mae_scorer(estimator, X, y):
    predictions = estimator.predict(X)
    mae = np.mean(np.abs(predictions - y))
    return mae

# Create a custom scorer
mae_scorer = make_scorer(custom_mae_scorer, greater_is_better=False)

# Define the parameter grid
param_grid = {
    'n_layers': [2, 3, 4],
    'd_model': [256, 512, 1024],
    'dropout': [0.1, 0.2, 0.3],
    'learning_rate': [1e-4, 1e-3, 1e-2]
}

# Initialize the model
saits = SAITS(
    n_steps=data_normalized.shape[1],
    n_features=data_normalized.shape[2],
    n_layers=2,
    d_model=512,
    d_ffn=512,
    n_heads=8,
    d_k=64,
    d_v=64,
    dropout=0.1,
    attn_dropout=0.1,
    diagonal_attention_mask=True,
    ORT_weight=1,
    MIT_weight=1,
    batch_size=5,
    epochs=10, 
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=device, 
    saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model",
    model_saving_strategy="best",


    
)

# Perform grid search
grid_search = GridSearchCV(estimator=saits, param_grid=param_grid, cv=3)
grid_search.fit(train_data, val_data)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")


SyntaxError: invalid non-printable character U+00A0 (<ipython-input-25-838d13c7a70f>, line 8)

In [ ]:

from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV


# Custom scoring function
def custom_mae_scorer(estimator, X, y):
    predictions = estimator.predict(X)
    mae = np.mean(np.abs(predictions - y))
    return mae

# Create a custom scorer
mae_scorer = make_scorer(custom_mae_scorer, greater_is_better=False)

# Define the parameter grid
param_grid = {
    'n_layers': [2, 3, 4],
    'd_model': [256, 512, 1024],
    'dropout': [0.1, 0.2, 0.3],
    'learning_rate': [1e-4, 1e-3, 1e-2]
}

# Initialize the model
saits = SAITS(
    n_steps=data_normalized.shape[1],
    n_features=data_normalized.shape[2],
    n_layers=2,
    d_model=512,
    d_ffn=512,
    n_heads=8,
    d_k=64,
    d_v=64,
    dropout=0.1,
    attn_dropout=0.1,
    diagonal_attention_mask=True,
    ORT_weight=1,
    MIT_weight=1,
    batch_size=5,
    epochs=10, 
    patience=6,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=device, 
    saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model",
    model_saving_strategy="best",


    
)

# Perform grid search
grid_search = GridSearchCV(estimator=saits, param_grid=param_grid, cv=3)
grid_search.fit(train_data, val_data)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")


SyntaxError: invalid non-printable character U+00A0 (<ipython-input-26-004ee1d3b0cf>, line 7)